<a href="https://colab.research.google.com/github/javiimo/ImageClassificationAssignment/blob/main/CLIPClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
! pip install datasets transformers


import clip
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import copy





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.9 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-meoqy32g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-meoqy32g
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [ ]:
imagenetv2= 'vaishaal/ImageNetV2' #https://github.com/modestyachts/ImageNetV2?tab=readme-ov-file
class HFDataset:
    def __init__(self, name='barkermrl/imagenet-a', split=None, config=None):
        self.name = name
        self.dataset = None
        self.datasetdict = None #In case there are multiple splits and none is selected
        self.split = split


    def print_info(self):
        dset.inspect()
        print('Split types:')
        print(dset.split_types())
        print('Configurations:')
        print(self.config_names())


    def inspect(self):
        from datasets import load_dataset_builder
        ds_builder = load_dataset_builder(self.name)
        print('Description:')
        print(ds_builder.info.description)
        print('Features: ')
        print(ds_builder.info.features)

    def split_types(self):
        from datasets import get_dataset_split_names
        return get_dataset_split_names(self.name) #list of split types

    def config_names(self):
        from datasets import get_dataset_config_names
        return get_dataset_config_names(self.name) #list of subdataset

    def load(self, split=None, config=None):
        # Set config and split to instance attributes if provided
        if config is not None:
            self.config = config
        if split is not None:
            self.split = split

        # Choose the appropriate dataset loading mechanism
        if self.split is None:
            # If no split is defined, it implies a full dataset dict might be loaded
            if self.config is not None:
                print('Setting a DatasetDict')
                self.datasetdict = load_dataset(self.name, self.config)
            else:
                print('Setting a DatasetDict')
                self.datasetdict = load_dataset(self.name)
        else:
            # Load specific split
            if self.config is not None:
                self.dataset = load_dataset(self.name, self.config, split=self.split)
            else:
                self.dataset = load_dataset(self.name, split=self.split)




In [ ]:
dset = HFDataset(name=imagenetv2)
dset.print_info()
#dset.load(split='train')
#dset.dataset[900]


Description:

Features: 
None
Split types:
['train']
Configurations:
['default']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision import datasets
imagenetv2 = datasets.ImageFolder(root='/content/drive/MyDrive/Petaloso Project/Code/Datasets/imagenetv2')
imageneta = datasets.ImageFolder(root='/content/drive/MyDrive/Petaloso Project/Code/Datasets/imagenet-a')

In [ ]:
# Load our data sets
from datasets import load_dataset

dataset_imageneta = load_dataset('barkermrl/imagenet-a',  split="test")
#dataset_imagenetv2 = load_dataset('imagenet_v2')


In [ ]:

class CLIPModel:
    def __init__(self, model_name='ViT-B/32', device=None):
        self.device = device if device else "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load(model_name, self.device)
        #self.model = self.convert_model_parameters_to_float32(self.model)
        #self.optimizer = optim.Adam(self.model.parameters(), lr=0.000000001, weight_decay=2e-4) #Numerical inestability
        self.optimizer = optim.SGD(self.model.parameters(), lr=1000, momentum=0.9)
        self.text_features = None
        self.requiring_grads = None

    def require_CLIP_gradients(self, state = True):
        if self.requiring_grads is None or state != self.requiring_grads: #don't change if the state is already OK
            for param in self.model.parameters():
                param.requires_grad = state
            self.requiring_grads = state

    def convert_model_parameters_to_float32(self, model):
        for param in model.parameters():
            param.data = param.data.to(torch.float32)
        return model

    def load_data(self):
        cifar100 = torchvision.datasets.CIFAR100(root='./data', download=True, train=False)
        return cifar100

    #This are heuristic labels
    def tokenize_labels(self, classes):
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classes]).to(self.device)
        with torch.no_grad():
            self.text_features = self.model.encode_text(text_inputs)
            self.text_features /= self.text_features.norm(dim=-1, keepdim=True)

    def augment_image(self, image, num_augmentations=100):
        augmentations = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=30),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0), ratio=(0.75, 1.333)),
        ])
        augmented_images = [self.preprocess(image).unsqueeze(0).to(self.device)]
        for _ in range(num_augmentations):
            augmented_images.append(self.preprocess(augmentations(image)).unsqueeze(0).to(self.device))
        batch = torch.vstack(augmented_images)
        return batch

    def compute_entropy(self, x):
        log_x = torch.log2(x.clamp_min(1e-20))
        entropy = -torch.sum(x * log_x)
        return entropy

    def class_probabilities(self, text_features, image_features):
        #Compute cosine similarities
        return (image_features @ text_features.T).softmax(dim=-1)

    def confidence_selection(self, similarities, percentile = 0.8):
        entropies = torch.tensor([self.compute_entropy(row) for row in similarities])
        sorted_entropies, _ = torch.sort(entropies, descending=True)
        threshold = sorted_entropies[int(len(sorted_entropies) * percentile)]
        boolean_mask = entropies < threshold
        return similarities[boolean_mask]

    def entropy_loss_MEMO(self, image_features, text_features = None, conf_sel=False):
        if text_features is None:
            text_features = self.text_features
        similarities = self.class_probabilities(text_features, image_features)
        # Apply confidence selection to rule out high entropy augmentations
        if conf_sel:
            similarities = self.confidence_selection(similarities)
        # Compute the entropy of every text caption accross all augmentations
        entropies = [self.compute_entropy(row) for row in similarities]
        return torch.stack(entropies).mean()

    def entropy_loss_TPT(self, image_features, text_features = None):
        if text_features is None:
            text_features = self.text_features
        similarities = self.class_probabilities(text_features, image_features)
        # Confidence selection for the augmented views:
        similarities = self.confidence_selection(similarities)
        # Average the caption probabilities across all augmentations
        avg_probs = torch.tensor([row.mean() for row in similarities.T])
        # Compute the entropy of the averaged probability distribution
        return self.compute_entropy(avg_probs), avg_probs

    def grad_descent_step(self, loss):
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def predict(self, image):
        self.model.eval()
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            image_features = self.model.encode_image(image)
            norms = image_features.norm(dim=-1, keepdim=True)
            if (norms == 0).any():
                print("Zero norm found in image features")
            image_features = image_features / norms.clamp_min(1e-10)

        similarity = self.class_probabilities(self.text_features, image_features)
        prediction = torch.argmax(similarity).item()
        entropy = float(self.compute_entropy(similarity))
        return prediction, similarity, entropy

    def MEMO(self, image, num_augmentations=100, conf_sel = False):
        # Save original parameters
        original_params = {name: param.clone() for name, param in self.model.named_parameters()}

        # Require gradients to update the CLIP parameters
        self.require_CLIP_gradients(state = True)
        try:
            self.model.train() #DO WE WANT BATCH NORMALIZ AND DROPOUT?
            batch = self.augment_image(image, num_augmentations)
            image_features = self.model.encode_image(batch)
            norms = image_features.norm(dim=-1, keepdim=True)
            if (norms == 0).any():
                print("Zero norm found in image features")
            image_features = image_features / norms.clamp_min(1e-10)

            loss = self.entropy_loss_MEMO(image_features, conf_sel= conf_sel)
            self.grad_descent_step(loss)

            if any(torch.isnan(param).any() for param in self.model.parameters()):
                print("nan values detected in model parameters after updating")
            # Predict using the updated model
            prediction, similarity, entropy = self.predict(image)
        finally:
            # Restore original parameters
            with torch.no_grad():
                for name, param in self.model.named_parameters():
                    param.copy_(original_params[name])
        return prediction, similarity, entropy

    def TPT(self, image, num_augmentations=100):
        batch = self.augment_image(image, num_augmentations)
        image_features = self.model.encode_image(batch)
        norms = image_features.norm(dim=-1, keepdim=True)
        if (norms == 0).any():
            print("Zero norm found in image features")
        image_features = image_features / norms.clamp_min(1e-10)

        entropy, avg_probs = self.entropy_loss_TPT(image_features)
        prediction = torch.argmax(avg_probs).item()
        return prediction, avg_probs, float(entropy)

    def train_CoOp(self):
        #Prevent CLIP parameters from changing
        self.require_CLIP_gradients(state=False)


# Preparing the class for usage
clip_model = CLIPModel()

In [ ]:
#Loading CIFAR100 for one image tries_
cifar100 = clip_model.load_data()
clip_model.tokenize_labels(cifar100.classes)
image, class_id = cifar100[3637]

Files already downloaded and verified


In [ ]:
# Prediction using CLIP out of the box
prediction1, similarity1, entropy1 = clip_model.predict(image)
print(similarity1, prediction1, entropy1)

tensor([[0.0101, 0.0102, 0.0102, 0.0099, 0.0099, 0.0100, 0.0100, 0.0103, 0.0098,
         0.0100, 0.0101, 0.0100, 0.0097, 0.0098, 0.0101, 0.0099, 0.0101, 0.0098,
         0.0102, 0.0101, 0.0099, 0.0101, 0.0099, 0.0098, 0.0100, 0.0099, 0.0103,
         0.0104, 0.0100, 0.0102, 0.0096, 0.0100, 0.0103, 0.0098, 0.0096, 0.0100,
         0.0101, 0.0098, 0.0098, 0.0099, 0.0101, 0.0103, 0.0103, 0.0100, 0.0104,
         0.0099, 0.0100, 0.0098, 0.0098, 0.0100, 0.0103, 0.0102, 0.0100, 0.0100,
         0.0099, 0.0099, 0.0100, 0.0099, 0.0099, 0.0101, 0.0100, 0.0101, 0.0100,
         0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0096, 0.0096, 0.0099, 0.0098,
         0.0101, 0.0098, 0.0101, 0.0099, 0.0097, 0.0103, 0.0107, 0.0102, 0.0098,
         0.0098, 0.0100, 0.0104, 0.0099, 0.0102, 0.0101, 0.0099, 0.0101, 0.0098,
         0.0099, 0.0100, 0.0099, 0.0106, 0.0098, 0.0097, 0.0098, 0.0098, 0.0100,
         0.0103]]) 78 6.643564224243164


In [ ]:
#Prediction using MEMO at test time
prediction2, similarity2, entropy2 = clip_model.MEMO(image, num_augmentations=100)
print(similarity2, prediction2, entropy2)

tensor(6.6438, grad_fn=<MeanBackward0>)
tensor([[0.0103, 0.0099, 0.0103, 0.0100, 0.0094, 0.0099, 0.0101, 0.0100, 0.0098,
         0.0099, 0.0101, 0.0102, 0.0096, 0.0101, 0.0100, 0.0097, 0.0098, 0.0099,
         0.0105, 0.0102, 0.0100, 0.0098, 0.0097, 0.0097, 0.0095, 0.0099, 0.0096,
         0.0101, 0.0102, 0.0101, 0.0095, 0.0097, 0.0095, 0.0106, 0.0096, 0.0101,
         0.0099, 0.0100, 0.0094, 0.0099, 0.0099, 0.0106, 0.0101, 0.0102, 0.0101,
         0.0096, 0.0102, 0.0107, 0.0099, 0.0097, 0.0099, 0.0100, 0.0108, 0.0100,
         0.0100, 0.0095, 0.0108, 0.0101, 0.0104, 0.0109, 0.0103, 0.0103, 0.0103,
         0.0099, 0.0097, 0.0098, 0.0096, 0.0094, 0.0098, 0.0100, 0.0103, 0.0098,
         0.0095, 0.0096, 0.0097, 0.0101, 0.0097, 0.0102, 0.0104, 0.0099, 0.0098,
         0.0100, 0.0103, 0.0105, 0.0101, 0.0102, 0.0101, 0.0100, 0.0101, 0.0105,
         0.0099, 0.0098, 0.0103, 0.0102, 0.0096, 0.0096, 0.0105, 0.0099, 0.0102,
         0.0103]]) 59 6.643089294433594


In [ ]:
# Prediction using TPT
prediction3,prob_avg, entropy3 = clip_model.TPT(image)
print(entropy3)
print(prob_avg)
print(prediction3)

6.643748760223389
tensor([0.0102, 0.0099, 0.0101, 0.0101, 0.0100, 0.0099, 0.0100, 0.0101, 0.0099,
        0.0100, 0.0101, 0.0100, 0.0098, 0.0099, 0.0100, 0.0099, 0.0101, 0.0099,
        0.0101, 0.0101, 0.0099, 0.0100, 0.0099, 0.0100, 0.0099, 0.0099, 0.0101,
        0.0102, 0.0100, 0.0102, 0.0098, 0.0099, 0.0101, 0.0100, 0.0097, 0.0100,
        0.0100, 0.0099, 0.0098, 0.0100, 0.0100, 0.0101, 0.0101, 0.0100, 0.0101,
        0.0099, 0.0100, 0.0101, 0.0099, 0.0101, 0.0100, 0.0102, 0.0102, 0.0101,
        0.0099, 0.0098, 0.0102, 0.0101, 0.0099, 0.0103, 0.0101, 0.0101, 0.0101,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0099, 0.0097, 0.0100, 0.0100, 0.0098,
        0.0101, 0.0099, 0.0099, 0.0100, 0.0098, 0.0101, 0.0102, 0.0099, 0.0099,
        0.0098, 0.0101, 0.0103, 0.0099, 0.0101, 0.0099, 0.0099, 0.0100, 0.0100,
        0.0099, 0.0101, 0.0100, 0.0103, 0.0097, 0.0100, 0.0101, 0.0100, 0.0100,
        0.0102])
59


In [ ]:
#Prediction using MEMO with confidence selection at test time
prediction4, similarity4, entropy4 = clip_model.MEMO(image, num_augmentations=100, conf_sel=True)
print(similarity4, prediction4, entropy4)

tensor(6.6437, grad_fn=<MeanBackward0>)
tensor([[0.0105, 0.0097, 0.0105, 0.0099, 0.0095, 0.0098, 0.0101, 0.0100, 0.0096,
         0.0099, 0.0104, 0.0105, 0.0093, 0.0096, 0.0101, 0.0097, 0.0099, 0.0095,
         0.0105, 0.0101, 0.0096, 0.0098, 0.0097, 0.0098, 0.0098, 0.0095, 0.0099,
         0.0101, 0.0102, 0.0103, 0.0098, 0.0097, 0.0100, 0.0100, 0.0096, 0.0103,
         0.0102, 0.0097, 0.0096, 0.0096, 0.0098, 0.0101, 0.0100, 0.0102, 0.0103,
         0.0098, 0.0104, 0.0103, 0.0097, 0.0096, 0.0101, 0.0104, 0.0103, 0.0104,
         0.0098, 0.0097, 0.0104, 0.0104, 0.0099, 0.0106, 0.0104, 0.0104, 0.0104,
         0.0098, 0.0098, 0.0099, 0.0094, 0.0098, 0.0094, 0.0100, 0.0105, 0.0100,
         0.0098, 0.0098, 0.0100, 0.0102, 0.0095, 0.0106, 0.0106, 0.0099, 0.0099,
         0.0097, 0.0104, 0.0106, 0.0101, 0.0099, 0.0100, 0.0097, 0.0101, 0.0099,
         0.0095, 0.0100, 0.0104, 0.0103, 0.0093, 0.0099, 0.0100, 0.0098, 0.0104,
         0.0105]]) 83 6.64306640625


In [ ]:
# Check if the similarities after the two MEMO implementations (with confidence selection and without it) are the same or not
s1 = similarity1.to(torch.float16)
s2 = similarity2.to(torch.float16)
s4 = similarity4.to(torch.float16)
print('Clip vs Memo without conf sel')
print(s1==s2)
print('Clip vs Memo with conf sel')
print(s1==s4)
print('Memo with conf sel vs Memo without conf sel')
print(s2==s4)

Clip vs Memo without conf sel
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])
Clip vs Memo with conf sel
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False, 

In [ ]:
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer

_tokenizer = _Tokenizer()

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection

    def forward(self, prompts, tokenized_prompts):
        x = prompts + self.positional_embedding
        x = x.permute(1, 0, 2)  # [batch_size, n_ctx, transformer.width] -> [n_ctx, batch_size, transformer.width]
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # [n_ctx, batch_size, transformer.width] -> [batch_size, n_ctx, transformer.width]
        x = self.ln_final(x)

        # Take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

        return x

In [ ]:
class PromptLearner(nn.Module):
    def __init__(self, clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        n_cls = len(classnames)
        ctx_dim = clip_model.ln_final.weight.shape[0]
        clip_imsize = clip_model.visual.input_resolution

        # Use given words to initialize context vectors
        # De aquí sacamos el context vector (tokenizado si lo sacamos de un cierto
        # texto o directamente un vector random que no sale de tokenizar
        # si lo iniciamos en plan random) y el prompt prefix que es el texto con
        # el que comenzamos antes de entrenar, que puede ser o texto inicial
        # con sentido o una X que no representa nada si no hay texto que inicialize
        # solo representa la cantidad de palabras que hay
        if ctx_init:
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init).to(clip_model.token_embedding.weight.device)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt)
            ctx_vectors = embedding[0, 1 : 1 + n_ctx, :]
            prompt_prefix = ctx_init
        else:
            if csc:
                print("Initializing class-specific contexts")
                ctx_vectors = torch.empty(n_cls, n_ctx, ctx_dim)
            else:
                print("Initializing a generic context")
                ctx_vectors = torch.empty(n_ctx, ctx_dim)

            torch.nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)

        print(f"Initial context: '{prompt_prefix}'")
        print(f"Number of context words (tokens): {n_ctx}")

        # These are the `prompts` we want to optimize
        self.ctx = nn.Parameter(ctx_vectors)

        classnames = [name.replace("_", " ") for name in classnames]
        name_lens = [len(_tokenizer.encode(name)) for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]

        # print("+++")
        # print("Prompts:")
        # for p in prompts:
        #     print(p)
        # print("+++")

        # Aqui está tokenizando a partir de el prompt (las X o lo que le hayamos
        # dado) pero no usa el context vector pa nada. Pero ese es el que nos importa
        # así que NO ENTIENDO ESTOS PA QUE SON. PARA SACAR EL SOS, CLS y EOS tokens
        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts]).to(clip_model.token_embedding.weight.device)

        with torch.no_grad():
            embedding = clip_model.token_embedding(tokenized_prompts)

        # These token vectors will be saved when in save_model(),
        # but they should be ignored in load_model() as we want to use
        # those computed using the current class names.
        # Buffer implica que no computas el gradiente para estos tokens. Asi que
        # son constantes!
        self.register_buffer("token_prefix", embedding[:, :1, :])  # SOS
        self.register_buffer("token_suffix", embedding[:, 1 + n_ctx :, :])  # CLS, EOS

        self.n_cls = n_cls
        self.n_ctx = n_ctx
        self.tokenized_prompts = tokenized_prompts
        self.name_lens = name_lens
        self.class_token_position = class_token_position

    def forward(self):
        prefix = self.token_prefix
        suffix = self.token_suffix
        ctx = self.ctx

        # If CoOp, expand the ctx for all classes (implying a shared context across all classes)
        if ctx.dim() == 2:
            ctx = ctx.unsqueeze(0).expand(self.n_cls, -1, -1)

        #Metemos el class token donde toque.
        #PERO AQUI NO ESTÁ EL CLASS TOKEN! SII VA EN EL SUFFIX JUNTO CON EOS
        if self.class_token_position == "end":
            prompts = torch.cat(
                [
                    prefix,  # (n_cls, 1, dim)
                    ctx,     # (n_cls, n_ctx, dim)
                    suffix,  # (n_cls, *, dim)
                ],
                dim=1,
            )

        elif self.class_token_position == "middle":
            half_n_ctx = self.n_ctx // 2
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i : i + 1, :, :]
                class_i = suffix[i : i + 1, :name_len, :]
                suffix_i = suffix[i : i + 1, name_len:, :]
                ctx_i_half1 = ctx[i : i + 1, :half_n_ctx, :]
                ctx_i_half2 = ctx[i : i + 1, half_n_ctx:, :]
                prompt = torch.cat(
                    [
                        prefix_i,     # (1, 1, dim)
                        ctx_i_half1,  # (1, n_ctx//2, dim)
                        class_i,      # (1, name_len, dim)
                        ctx_i_half2,  # (1, n_ctx//2, dim)
                        suffix_i,     # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        elif self.class_token_position == "front":
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i : i + 1, :, :]
                class_i = suffix[i : i + 1, :name_len, :]
                suffix_i = suffix[i : i + 1, name_len:, :]
                ctx_i = ctx[i : i + 1, :, :]
                prompt = torch.cat(
                    [
                        prefix_i,  # (1, 1, dim)
                        class_i,   # (1, name_len, dim)
                        ctx_i,     # (1, n_ctx, dim)
                        suffix_i,  # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        else:
            raise ValueError

        return prompts

In [ ]:
main_coop()

In [ ]:
class CoCoOp:
    def __init__(self, CLIP_model, device=None):
        self.device = device if device else "cuda" if torch.cuda.is_available() else "cpu"

TESTSSSS


In [ ]:
# Set the class names for imagenet-A
def classnames_imagenetA():
    # Define the path to the words file
    file_path = '/content/drive/MyDrive/Petaloso Project/Code/Datasets/words_imageneta.txt'

    # Initialize an empty list to store the class names
    class_names = []

    # Open and read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Split each line into wnid and class name, and strip to remove any leading/trailing whitespace
            parts = line.strip().split(' ', 1)
            if len(parts) > 1:
                # Append only the class name (second part) to the list
                class_names.append(parts[1])
    return class_names

In [ ]:
#Implementing subsets
import numpy as np
from torch.utils.data import Subset
def create_stratified_subset(dataset, num_samples_per_class=5):
    # Fix the random seed for reproducibility
    torch.manual_seed(0)
    np.random.seed(0)

    # Determine class indices
    targets = np.array([s[1] for s in dataset.samples])
    classes, class_indices = np.unique(targets, return_inverse=True)

    # Select samples from each class
    indices = []
    for c in classes:
        class_idx = np.where(class_indices == c)[0]
        if len(class_idx) >= num_samples_per_class:
            selected_indices = np.random.choice(class_idx, num_samples_per_class, replace=False)
            indices.extend(selected_indices)
        else:
            # If a class has fewer than the desired number, take all
            indices.extend(class_idx)

    # Create subset
    subset = Subset(dataset, indices)
    return subset

subset = create_stratified_subset(imageneta, num_samples_per_class=5)

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

imageneta = datasets.ImageFolder(root='/content/drive/MyDrive/Petaloso Project/Code/Datasets/imagenet-a')

def testing(dataset, model,method='CLIP', batch_size=32, num_aug=100):
    model.tokenize_labels(classnames_imagenetA())

    def custom_collate_fn(batch):
        # Extract images and labels from the batch
        images = [item[0] for item in batch]  # PIL images
        labels = [item[1] for item in batch]  # Corresponding labels
        return images, labels
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

    # Initialize lists to store results
    correct_predictions = 0
    total_predictions = 0
    entropies = []
    confidences = []

    # Evaluation loop
    for images, labels in tqdm(dataloader):
        for image, label in zip(images, labels):
            try:
                # We choose how the test time predictions are made
                if method == 'CLIP':
                    prediction, probs, entropy = model.predict(image)
                elif method == 'MEMO':
                    prediction, probs, entropy = model.MEMO(image, num_augmentations=num_aug)
                elif method == 'MEMO_CONF':
                    prediction, probs, entropy = model.MEMO(image, num_augmentations=num_aug, conf_sel = False)
                elif method == 'TPT':
                    prediction, probs, entropy = model.TPT(image, num_augmentations=num_aug)
                else:
                    print('Enter a valid method for testing.')

                if int(prediction) == int(label):
                    correct_predictions += 1
                total_predictions += 1
                entropies.append(entropy)
                confidences.append(torch.max(probs).item())
            except Exception as e:
                print(f"An error occurred: {e}")

    # Post evaluation statistics or processing
    accuracy = (correct_predictions / total_predictions) * 100
    average_entropy = sum(entropies) / len(entropies)
    average_confidence = sum(confidences) / len(confidences)
    print(f'Accuracy: {accuracy:.2f}%')
    print(f'Average entropy across all predictions: {average_entropy:.2f}')


testing(subset,clip_model, method = 'CLIP', batch_size=35)


100%|██████████| 29/29 [00:18<00:00,  1.57it/s]

Accuracy: 31.26%
Average entropy across all predictions: 7.64
